## Object Detection Pipeline on UCS using Tensorflow 2.0

This notebook focuses on implementing object detection as a Kubeflow pipeline on Cisco UCS by using Darknet which is a open-source neural network framework, YOLO (You Only Look Once) which is a real-time object detection system.

## Clone Cisco Kubeflow starter pack repository

In [ ]:
BRANCH_NAME="dev" #Provide git branch "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

## Install required packages

In [ ]:
!pip install kfp==1.0.1 pillow==7.2.0 mlflow==1.13.1 --user

## Restart kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import libraries

In [1]:
import os
import json
import time
import yaml
import calendar
import requests
import logging
import numpy as np
from PIL import Image, ImageDraw

#Kubeflow
import kfp
from kfp.aws import use_aws_secret
import kfp.compiler as compiler

#Kubernetes
from kubernetes import client

#Tensorflow
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

## Load pipeline components

Declare the paths of respective YAML configuration files of each of the pipeline components, in order to load each component into a variable for pipeline execution. 

In [2]:
path='cisco-kubeflow-starter-pack/apps/computer-vision/object-detection/onprem/pipeline/components/v2/'
component_root_train= path+'train/'
component_root_tensorboard= path+'tensorboard/'
component_root_inference= path+'inference/'

train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
infer_op = kfp.components.load_component_from_file(os.path.join(component_root_inference, 'component.yaml'))
tensorboard_op = kfp.components.load_component_from_file(os.path.join(component_root_tensorboard, 'component.yaml'))

## Define volume claim & volume mount for storage during pipeline execution

Persistent volume claim & volume mount is created for the purpose of storing entities such as dataset, model files, etc, and to share the stored resources between the various components of the pipeline during it's execution. 

In [3]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

## Define pipeline function

In [4]:
timestamp = str(calendar.timegm(time.gmtime()))
def object_detection_pipeline():
    # Defining Task for tensorboard
    tf_tensorboard_task = tensorboard_op(timestamp=timestamp)
    tf_tensorboard_task.add_volume(nfs_volume)
    tf_tensorboard_task.add_volume_mount(nfs_volume_mount)
    
    # Defining Task for Model Training
    tf_train_model_task = train_op(timestamp=timestamp)
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount) 
    tf_train_model_task.after(tf_tensorboard_task) 
    
    #Defining Task for Infer training
    tf_infer_model_task = infer_op(timestamp=timestamp)
    tf_infer_model_task.add_volume(nfs_volume)
    tf_infer_model_task.add_volume_mount(nfs_volume_mount) 
    tf_infer_model_task.after(tf_train_model_task)

## Compile pipeline function

Compile the pipeline function to create a tar ball for the pipeline.

In [5]:
# Compile pipeline
try:
    compiler.Compiler().compile(object_detection_pipeline, 'object-detection.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

## Create pipeline experiment

In [6]:
kp_client = kfp.Client()
EXPERIMENT_NAME = 'Object Detection'
experiment = kp_client.create_experiment(name=EXPERIMENT_NAME)

## Initialize pipeline parameters & run pipeline

In [7]:
run_name = 'tensorflow-object-detection-'+timestamp

# Execute pipeline
run = kp_client.run_pipeline(experiment.id, run_name,'object-detection.tar.gz', 
                          params={})

## Retrieve current pipeline run ID

In [ ]:
run_id = str(run.id)
run_id

## Delete pipeline run

In [ ]:
kp_client.runs.delete_run(run_id)
print("Pipeline run with run ID '%s' successfully deleted"%run_id)